In [1]:
## Imports ##
#------------------------------------------------#
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.autograd as autograd
#------------------------------------------------#
import matplotlib.pyplot as plt
import numpy as np
from time import time
import math
import os
import sys
from tqdm import tqdm_notebook as tqdm
#------------------------------------------------#
from data import Dataset
from train_isola import SiameseNetwork

Device mode:  cuda


/home/gefenkohavi/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def rmd(img): # ReMove Depth channel
    return img[:,:3,:,:]

In [6]:
is_depth = False

if is_depth:
    model = torch.load('model_isola_onlydepth').cuda()
else:    
    model = torch.load('model_isola_nodepth').cuda()

#Testing
batch_size = 256
accuracy = []
trainset = Dataset(train=False, split_size=.8)
trainloader = DataLoader(dataset=trainset, batch_size=batch_size)

/home/gefenkohavi/anaconda3/lib/python3.6/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'train_isola.SiameseNetwork' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/gefenkohavi/anaconda3/lib/python3.6/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/gefenkohavi/anaconda3/lib/python3.6/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by ac

In [8]:
model.eval()
with torch.no_grad():
    for epoch in tqdm(range(5)):
        for i, (a, b, c) in tqdm(enumerate(trainloader)):
            a, b, c = (a.permute(0,3,1,2).cuda().type(torch.float32), 
                       b.permute(0,3,1,2).cuda().type(torch.float32), c.cuda().type(torch.float))
            if is_depth:
                # model(a, b).squeeze(1)
                pred_c = model(a[:,-1:,:,:], b[:,-1:,:,:]).squeeze(1)
            else: 
                pred_c = model(rmd(a), rmd(b)).squeeze(1)
            pred_c = list(torch.round(pred_c).cpu().numpy())
            c = list(c.cpu().numpy())
            accuracy.append(sum([1 for x, y in zip(pred_c, c) if x == y]) / len(c))
    print('Accuracy:', sum(accuracy) / len(accuracy))


Accuracy: 0.7823745265151514
